In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [3]:
# Load data
data = pd.read_excel("/content/Base de Données MATHURANCE.xlsm")

In [4]:
# Clean column names
data.columns = data.columns.str.lower().str.replace(' ', '_')

In [5]:
# Convert date_survenance to datetime
data['date_survenance'] = pd.to_datetime(data['date_survenance'], format='%d/%m/%Y')

In [8]:
data.columns

Index(['exercice_', 'branche', 'code_produit', 'désignation_produit',
       'sous-branche', 'date_survenance', 'règlement'],
      dtype='object')

In [9]:
# Group by exercice and arrange by date_survenance
data = data.sort_values(by=['exercice_', 'date_survenance'])

In [10]:

# Create year_of_sinistre column
data['year_of_sinistre'] = data['date_survenance'].dt.year

In [14]:
# Select relevant columns
boop = data[['exercice_', 'date_survenance', 'sous-branche', 'règlement', 'year_of_sinistre']]

In [15]:
# Remove duplicates
boop_clean = boop.drop_duplicates()

In [17]:
# Create development_year column
boop_clean['development_year'] = boop_clean['exercice_'] - boop_clean['year_of_sinistre']

<ipython-input-17-b94d289fc2ac>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boop_clean['development_year'] = boop_clean['exercice_'] - boop_clean['year_of_sinistre']


In [19]:
# Summarize total_reglement by year_of_sinistre and development_year
clean_data = boop_clean.groupby(['year_of_sinistre', 'development_year'])['règlement'].sum().unstack(fill_value=0)

In [20]:
# First branch: Responsabilité Civile
responsabilite_civile = boop_clean[boop_clean['sous-branche'] == "Responsabilité Civile"]
responsabilite_civile = responsabilite_civile.groupby(['year_of_sinistre', 'development_year'])['règlement'].sum().unstack(fill_value=0)

In [22]:
# Load filled data
responsabilite_civile_filled = pd.read_csv("respociviletrain.csv")

In [24]:
responsabilite_civile_filled

,Year,Development Year,Claim Amount
0,1999,0,1.012859e+07
1,1999,1,1.807241e+08
2,1999,2,2.131659e+07
3,1999,3,1.191443e+08
4,1999,4,1.287867e+07
...,...,...,...
279,2021,1,5.688582e+07
280,2021,2,4.596211e+07
281,2022,0,5.954270e+07
282,2022,1,2.012872e+08


In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import mean_squared_error

# Load the training data
responsabilite_civile_filled = pd.read_csv("respociviletrain.csv")

# Split the data into training and testing sets
train_data, test_data = train_test_split(
    responsabilite_civile_filled,
    test_size=0.1,
    stratify=responsabilite_civile_filled['Claim Amount'],
    random_state=42  # For reproducibility
)

# Define the model pipeline
model_pipeline = Pipeline([
    ('log_transform', FunctionTransformer(np.log1p)),  # Apply log transformation
    ('model', RandomForestRegressor(random_state=42))  # Random Forest model
])

# Fit the model on the training data
model_pipeline.fit(
    train_data.drop(columns=['Claim Amount']),  # Features
    train_data['Claim Amount']  # Target variable
)

# Predict on the test set
test_predictions = model_pipeline.predict(
    test_data.drop(columns=['Claim Amount'])
)

# Calculate RMSE and MSE
rmse = np.sqrt(mean_squared_error(test_data['Claim Amount'], test_predictions))
mse = mean_squared_error(test_data['Claim Amount'], test_predictions)

print(f"Test RMSE: {rmse}")
print(f"Test MSE: {mse}")

# Load the real test data
responsabilite_real_test = pd.read_csv("respociviletest.csv")[['Development Year', 'Claim Amount']]

# Predict on the real test data
real_predictions = model_pipeline.predict(
    responsabilite_real_test.drop(columns=['Claim Amount'])
)

# Add predictions to the real test data
responsabilite_real_test['Predicted Claim Amount'] = real_predictions

# Combine the original training data with the real test data
combined_data = pd.concat([responsabilite_civile_filled, responsabilite_real_test], ignore_index=True)

# Pivot the data to create a wider format
final_data = combined_data.pivot(
    index='year_of_sinistre',  # Rows
    columns='Development Year',  # Columns
    values='Predicted Claim Amount'  # Values
)

# Display the final pivoted data
print(final_data)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [31]:
# Split data
train, test = train_test_split(responsabilite_civile_filled, test_size=0.25)

In [32]:

# Define model
model = RandomForestRegressor()

In [33]:
# Define pipeline
pipeline = Pipeline([
    ('log_transform', FunctionTransformer(np.log1p)),
    ('model', model)
])

In [34]:
# Fit model
pipeline.fit(train.drop(columns=['Claim Amount']), train['Claim Amount'])

Pipeline(steps=[('log_transform', FunctionTransformer(func=<ufunc 'log1p'>)),
                ('model', RandomForestRegressor())])

In [52]:
import pickle

# Save the model to a file
with open('responsibilitycivilemodel.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

In [35]:
# Predict on test set
predictions = pipeline.predict(test.drop(columns=['Claim Amount']))

In [53]:
# Calculate RMSE
rmse = np.sqrt(mean_squared_error(test['Claim Amount'], predictions))
mse = mean_squared_error(test['Claim Amount'], predictions)

In [54]:
mse

1.2843982891737128e+16

In [43]:
responsabilite_real_test = pd.read_csv("respociviletest.csv")

In [44]:
responsabilite_real_test

,Year,Development Year,Claim Amount
0,2001,20,NaN
1,2002,19,NaN
2,2002,20,NaN
3,2004,20,NaN
4,2005,13,NaN
...,...,...,...
215,2023,16,NaN
216,2023,17,NaN
217,2023,18,NaN
218,2023,19,NaN


In [55]:

# Predict on real test data
real_predictions = pipeline.predict(responsabilite_real_test.drop(columns=['Claim Amount']))

In [56]:

# Combine predictions with real test data
responsabilite_real_test['Claim Amount'] = real_predictions

In [57]:
# Pivot wider
final_data = pd.concat([responsabilite_civile_filled, responsabilite_real_test]).pivot(index='Year', columns='Development Year', values='Claim Amount')


In [63]:
incendie

,exercice_,date_survenance,sous-branche,règlement,year_of_sinistre,development_year
62490,2014,2004-03-12,Incendie,4987.96,2004,10
75708,2014,2004-04-10,Incendie,2748.92,2004,10
43966,2014,2004-08-09,Incendie,9767.31,2004,10
54308,2014,2004-08-12,Incendie,6511.47,2004,10
30088,2014,2004-08-14,Incendie,20488.14,2004,10
...,...,...,...,...,...,...
32631,2023,2023-12-19,Incendie,17189.61,2023,0
30488,2023,2023-12-20,Incendie,20000.00,2023,0
21271,2023,2023-12-21,Incendie,45238.33,2023,0
22410,2023,2023-12-27,Incendie,40830.00,2023,0


In [64]:
incendie = boop_clean[boop_clean['sous-branche'] == "Incendie"]
incendie = incendie.groupby(['year_of_sinistre', 'development_year'])['règlement'].sum().unstack(fill_value=0)

In [65]:
# Load filled data
incendie_filled = pd.read_csv("Incendietrain.csv")

In [69]:
train, test = train_test_split(incendie_filled, test_size=0.25)

In [70]:
# Fit model
pipeline.fit(train.drop(columns=['Claim Amount']), train['Claim Amount'])

Pipeline(steps=[('log_transform', FunctionTransformer(func=<ufunc 'log1p'>)),
                ('model', RandomForestRegressor())])

In [71]:
import pickle
# Save the model to a file
with open('incendiemodel.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

In [72]:
# Predict on test set
predictions = pipeline.predict(test.drop(columns=['Claim Amount']))

In [74]:
# Calculate RMSE
rmse = np.sqrt(mean_squared_error(test['Claim Amount'], predictions))
mse = mean_squared_error(test['Claim Amount'], predictions)

In [75]:
mse

1.7956111848432074e+17

In [ ]:
# Predict on real test data
incendie_real_test = pd.read_csv("Incendietest.csv")[['Development Year', 'Claim Amount']]
real_predictions = pipeline.predict(incendie_real_test.drop(columns=['Claim Amount']))

In [ ]:
# Combine predictions with real test data
incendie_real_test['Claim Amount'] = real_predictions

In [ ]:
# Pivot wider
final_data = pd.concat([incendie_filled, incendie_real_test]).pivot(index='year_of_sinistre', columns='Development Year', values='Claim Amount')

In [76]:
boop_clean.columns

Index(['exercice_', 'date_survenance', 'sous-branche', 'règlement',
       'year_of_sinistre', 'development_year'],
      dtype='object')

In [78]:
# Third branch: Risque simple
risk = boop_clean[boop_clean['sous-branche'] == "Risque simple"]
risk = risk.groupby(['year_of_sinistre', 'development_year'])['règlement'].sum().unstack(fill_value=0)

In [79]:
# Load filled data
risk_filled = pd.read_csv("risquesimpletrain.csv")

In [80]:
# Split data
train, test = train_test_split(risk_filled, test_size=0.25)

In [81]:


# Define model
model = RandomForestRegressor()

In [82]:

# Fit model
pipeline.fit(train.drop(columns=['Claim Amount']), train['Claim Amount'])

Pipeline(steps=[('log_transform', FunctionTransformer(func=<ufunc 'log1p'>)),
                ('model', RandomForestRegressor())])

In [84]:
import pickle
# Save the model to a file
with open('risquesimple.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

In [83]:

# Predict on test set
predictions = pipeline.predict(test.drop(columns=['Claim Amount']))

In [ ]:
# Calculate RMSE
rmse = np.sqrt(mean_squared_error(test['Claim Amount'], predictions))
mse = mean_squared_error(test['Claim Amount'], predictions))

In [ ]:
# Predict on real test data
risk_real_test = pd.read_csv("risquesimpletest.csv")[['Development Year', 'Claim Amount']]
real_predictions = pipeline.predict(risk_real_test.drop(columns=['Claim Amount']))

In [ ]:
# Combine predictions with real test data
risk_real_test['Claim Amount'] = real_predictions

In [ ]:
# Pivot wider
final_data = pd.concat([risk_filled, risk_real_test]).pivot(index='year_of_sinistre', columns='Development Year', values='Claim Amount')

In [87]:
boop_clean.columns

Index(['exercice_', 'date_survenance', 'sous-branche', 'règlement',
       'year_of_sinistre', 'development_year'],
      dtype='object')

In [88]:

cat = boop_clean[boop_clean['sous-branche'] == "CAT-NAT"]
cat = cat.groupby(['year_of_sinistre', 'development_year'])['règlement'].sum().unstack(fill_value=0)

In [89]:

# Load filled data
cat_filled = pd.read_csv("catnattrain.csv")

In [90]:

# Split data
train, test = train_test_split(cat_filled, test_size=0.25)

In [91]:
# Define model
model = ElasticNet(alpha=0.0001, l1_ratio=0.5)

In [92]:

# Fit model
pipeline.fit(train.drop(columns=['Claim Amount']), train['Claim Amount'])

Pipeline(steps=[('log_transform', FunctionTransformer(func=<ufunc 'log1p'>)),
                ('model', RandomForestRegressor())])

In [93]:
import pickle
# Save the model to a file
with open('catnat.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

In [94]:
# Predict on test set
predictions = pipeline.predict(test.drop(columns=['Claim Amount']))

In [96]:
 #Calculate RMSE
rmse = np.sqrt(mean_squared_error(test['Claim Amount'], predictions))
mse = mean_squared_error(test['Claim Amount'], predictions)

In [ ]:
cat_real_test = pd.read_csv("catnattest.csv")

In [99]:
# Predict on real test dat
real_predictions = pipeline.predict(cat_real_test.drop(columns=['Claim Amount']))

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- Year


In [ ]:
# Combine predictions with real test data
cat_real_test['Claim Amount'] = real_predictions

In [ ]:
# Pivot wider
final_data = pd.concat([cat_filled, cat_real_test]).pivot(index='year_of_sinistre', columns='Development Year', values='Claim Amount')

In [ ]:
# Combine metrics
model_metrics = pd.DataFrame({
    'category': ["Responsabilité civile", "Incendie", "Risque simple", "CAT-NAT"],
    'Model': ["Random Forest", "Random Forest", "Random Forest", "Linear regression"],
    'mse': [responsabilite_metrics['mse'], incendie_metrics['mse'], risk_metrics['mse'], cat_metrics['mse']]
})

print(model_metrics)